# Return Stats

## Import Libraries

In [ ]:
import os
os.chdir('..')

import pandas as pd
import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics.util import get_df_weights
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual

new_strat = False
notional_weights = []
weighted = [True, False]

CORR_LIST = ['full', 'equity_down', 'equity_up', 'fi_down', 'fi_up']

## Import Data

## Equity & FI data

In [ ]:
#Equity & FI data
asset_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\Historical Returns.xls', sheet_name='Historical Returns')
asset_df.columns = ['Name', 'Account Id', 'Return Type', 'Date',
       'Market Value', 'Return']
asset_ret = asset_df.pivot_table(values='Return', index='Date', columns='Name')
asset_ret /= 100
eq_fi_df = asset_ret[['Total EQ w/o Derivatives', 'Total Fixed Income']]
eq_fi_df.columns = ['Equity', 'Fixed Income']

## Benchmark data

In [ ]:
#benchmark data
bmk_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\bmk_data.xlsx', sheet_name='data_m', index_col=0)
bmk_df = bmk_df[['HFRXM Index','HFRIMI Index',
                  'SGIXTFXA Index', 'NEIXCTAT Index',
                 'HFRXAR Index', 'HFRXEW Index', 
                 'HEDGGLMA Index', 'CSLABT18 Index']]
#rename columns
bmk_df.columns = ['HFRX Macro/CTA', 'HFRI Macro Total Index',
                  'SGI Cross Asset Trend', 'SG Trend Index',
                  'HFRX Abs Ret', 'HFRX Equal Wgtd', 
                  'CS Global Macro','CS Managed Fut-18% vol']

bmk_df = dm.format_data(bmk_df)

In [ ]:
#group indicies into sub portfolios
sp_ind_dict = {'Global Macro':['HFRX Macro/CTA', 'HFRI Macro Total Index','CS Global Macro'],
            'Absolute Return':['HFRX Abs Ret', 'HFRX Equal Wgtd'],
                'Trend Following':['SGI Cross Asset Trend', 'SG Trend Index','CS Managed Fut-18% vol']}
#create bmk_dict with sub portfolios as keys
bmk_dict = {}
for key in sp_ind_dict:
    bmk_dict[key] = bmk_df[sp_ind_dict[key]].copy()

## Liquid Alts data

In [ ]:
#liq_alts data
liq_alts_df = pd.read_excel(dm.RETURNS_DATA_FP + 'liq_alts\\mgr_data_m.xlsx', sheet_name='bny',index_col=0)
liq_alts_sp = liq_alts_df[['Total Liquid Alts',  'Global Macro',  'Trend Following','Alt Risk Premia']]
liq_alts_sp = liq_alts_sp.iloc[52:,]
liq_alts_mgr = liq_alts_df[['1907 ARP EM', '1907 ARP TF', '1907 CFM ISE Risk Premia','1907 Campbell TF', '1907 III CV',
                            '1907 III Class A', '1907 Kepos RP','1907 Penso Class A', '1907 Systematica TF',
                            'ABC Reversion','Acadian Commodity AR', 'Black Pearl RP','Blueshift', 'Bridgewater Alpha',
                            'DE Shaw Oculus Fund', 'Duality','Element Capital', 'Elliott', 'One River Trend']]

In [ ]:
sp_mgr_dict = {'Global Macro':['1907 Penso Class A', 'Bridgewater Alpha','DE Shaw Oculus Fund', 'Element Capital', 'Elliott'],
                'Absolute Return':['1907 ARP EM','1907 III Class A', '1907 Kepos RP',
                            'ABC Reversion','Blueshift', 'Duality'],
                'Trend Following':['1907 ARP TF','1907 Campbell TF', '1907 Systematica TF', 'One River Trend', 'Trend Following']}

liq_alts_dict = {}
for key in sp_mgr_dict:
    liq_alts_dict[key] = liq_alts_df[sp_mgr_dict[key]].copy()
    liq_alts_dict[key].dropna(inplace=True)

# sp_df = liq_alts_df[['Total Liquid Alts']]

for key in liq_alts_dict:
    liq_alts_dict[key][key] = liq_alts_dict[key].sum(axis=1)/len(liq_alts_dict[key].columns)
#     sp_df = dm.merge_data_frames(sp_df,liq_alts_dict[key][[key]])


## Returns dict

In [ ]:
#Merge into one returns_dict
returns_dict = {}

for key in liq_alts_dict:
    temp_df = dm.merge_data_frames(liq_alts_dict[key].copy(),bmk_dict[key].copy())
    returns_dict[key] = dm.merge_data_frames(eq_fi_df.copy(),temp_df)
    

## Display Returns

In [ ]:
@interact
def display_returns(sub_port=returns_dict.keys()):
    return fmt.get_returns_styler(returns_dict[sub_port])

## Correlations

In [ ]:
#Compute Correlations
corr_dict = {}    
for key in returns_dict:
    temp_dict = summary.get_corr_analysis(returns_dict[key],include_fi=True)
    corr_dict[key] = temp_dict

#Display correlations
@interact
def display_correlations(sub_port=corr_dict.keys(),corr= CORR_LIST):
    corr_df = corr_dict[sub_port][corr][0]
    return plots.draw_heatmap(corr_df, half=False)

## Return Stats

In [ ]:
analytics_dict = {}
for key in returns_dict:
    temp_dict = summary.get_analysis(returns_dict[key], include_fi=True)
    analytics_dict[key] = temp_dict

#Display return stats
@interact
def display_analytics(sub_port = analytics_dict.keys()):
    return fmt.get_analytics_styler(analytics_dict[sub_port])

## Quintile Analysis

In [ ]:
@interact
def display_quintile_analysis(sub_port = returns_dict.keys()):    
    df = summary.get_grouped_data(returns_dict[sub_port])
    for col in df.columns:
        df[col] *= 100

    title ='Quintile Analysis'
    fig = px.bar(df,barmode='group',title = title)
    fig.update_layout(
        title = {'text':"<b>{}</b>".format(title),'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'},
        title_font_family = "Calibri",
        titlefont = {"size":24},
        xaxis = dict(tickfont = dict(size=12)),
        yaxis = dict(ticksuffix="%"),
        legend_title="Strategies",
        plot_bgcolor='White'
    )
    fig.update_xaxes(title_font_family = "Calibri",title_text = "<b>SPTR Monthly Returns Ranking</b>",
        title_font = {"size": 20},showline=True,linewidth=2,linecolor='black',mirror=False)
    fig.update_yaxes(title_font_family = "Calibri",title_text = "<b></b>",title_font = {"size": 20},
        showline=True,linewidth=2,linecolor='black',mirror=False)
    fig.show()

## Generate reports

In [ ]:
for key in returns_dict:
    rp.generate_strat_report(key,{'Monthly':returns_dict[key]}, freq_list=['Monthly'],include_fi=True)